# Chapter 8: Tree-Based Methods
## The Basics of Decision Trees
### Regression Trees
The general process(idea) of building a regression tree:
1. We divide the predictor space — that is, the set of possible values for $X_1, X_2, . . . , X_p$ — into $J$ distinct and non-overlapping regions, $R_1,R_2,...,R_J$.
2. For every observation that falls into the region $R_j$ , we make the same prediction, which is simply the mean of the response values for the training observations in $R_j$.

How to construct regions?
The goal is to find boxes $R_1, . . . , R_J$ that minimize the RSS, given by $\sum_{j=1}^{J} \sum_{i \in R_j} \left( y_i - \hat{y}_{R_j} \right)^2$ → computationally infeasible

**Recursive Binary Splitting** - a top-down *greedy* approach
- select the predictor $X_j$ and the cutpoint $s$ such that splitting the predictor space into the regions ${X|X_j < s}$ and ${X|X_j ≥ s}$ leads to the greatest possible reduction in RSS
- In greater detail, for any $j$ and $s$, we define the pair of half-planes $R_1(j, s) = {X|X_j < s}$ and $R_2(j, s) = {X|X_j ≥ s}$, and we seek the value of $j$ and $s$ that minimize the equation
$$
\sum_{i : x_i \in R_1(j,s)} \left( y_i - \hat{y}_{R_1} \right)^2
+ \sum_{i : x_i \in R_2(j,s)} \left( y_i - \hat{y}_{R_2} \right)^2
$$
- repeat the process, and split one of the two previously identified regions
- contibue the process until a stopping criterion is reached

**Prune Trees**

The process above is likely to overfit the data.

→ One alternative: to build the tree only so long as the decrease in the RSS due to each split exceeds some (high) threshold;
But, a seemingly worthless split early on in the tree might be followed by a very good split—that is, a split that leads to a large reduction in RSS later on.

A better strategy: to grow a very large tree $T_0$, and then *prune* it back in order to obtain a *subtree*.

How to decide the way to prune the tree? - Intuitively, to select a subtree that leads to the lowest test error rate.

cross-validation? - cumbersome

***Cost complexity pruning (weakest link pruning)*** - consider a sequence of trees indexed by a nonnegative tuning parameter $α$. For each value of $α$ there corresponds a subtree $T ⊂ T_0$ such that $\sum_{m=1}^{|T|} \sum_{i:x_i \in R_m} (y_i - \hat{y}_{R_m})^2 + \alpha |T|$ is as small as possible. $|T|$ is the number of terminal nodes of the tree $T$. The tuning parameter $α$ controls a trade-off between the subtree’s com- plexity and its fit to the training data.

---
**Algorithm: Build a Regression Tree**

1. Use *recursive binary splitting* to grow a large tree on the training data, stopping only when each terminal node has fewer than some minimum number of observations.
2. Apply *cost complexity pruning* to the large tree in order to obtain a sequence of best subtrees, as a function of $α$.
3. Use K-fold cross-validation to choose $α$. That is, divide the training observations into K folds. For each $k = 1, . . . , K$:
(a) Repeat Steps 1 and 2 on all but the kth fold of the training data.
(b) Evaluate the mean squared prediction error on the data in the left-out $k$th fold, as a function of $α$.
Average the results for each value of $α$, and pick $α$ to minimize the average error.
4. Return the subtree from Step 2 that corresponds to the chosen value of $α$.
---

### Classification Trees
In the classification setting, RSS cannot be used as a criterion for making the binary splits.

Alternatives:

- **Classification Error Rate**: $E=1-\max\limits_{k}(\hat{p_{mk}})$; the fraction of the training observations in that region that do not belong to the most common class; However, it turns out that classification error is not suﬀiciently sensitive for tree-growing.
- **Gini Index**: $G = \sum_k^K \hat{p_{mk}}(1-\hat{p_{mk}})$; a measure of node purity — a small value indicates that a node contains predominantly observations from a single class.
- **entropy**: $D=- \sum_k^K \hat{p_{mk}}log\hat{p_{mk}}$; similar to Gini Index numerically.

Any of these three approaches might be used when pruning the tree, but the classification error rate is preferable if prediction accuracy of the final pruned tree is the goal.

## Bagging, Random Forests, Boosting, and Bayesian Additive Regression Trees
By aggregating many decision trees, using ensmble methods like bagging, random forests, and boosting, the predictive performance of trees can be substantially improved.

An *ensemble method* is an approach that combines many simple “building block” models in order to obtain a single and potentially powerful model. 

### Bagging
Idea: Averaging a set of observations reduces variance. - calculate $\hat{f}^1(x),\hat{f}^2(x),...,\hat{f}^B(x)$ using B separate training sets, and average them in order to obtain a single low-variance statistical learning model, given by $\hat{f}_{avg}(x)=\frac{1}{B}\sum_{b=1}^B\hat{f}^b(x)$.

In pracctice, we can bootstrap by taking repeated samples from the (single) training data set. Then $\hat{f}_{bag}(x)=\frac{1}{B}\sum_{b=1}^B\hat{f}^{*b}(x)$

When applying bagging to decision trees, each of the B bootstrapped trees are not pruned, so each individual tree has high variance, but low bias.

In qualitative situations, we can record the class predicted by each of the B trees, and take a *majority vote*: the overall prediction is the most commonly occurring majority class among the B predictions.

*Variable Importance*: Although the collection of bagged trees is much more diﬀicult to interpret than a single tree, one can obtain an overall summary of the importance of each predictor using the RSS (for bagging regression trees) or the Gini index (for bagging classification trees). 

### Random Forests
As in bagging, we build a number of decision trees on bootstrapped training samples. But when building these decision trees, each time a split in a tree is considered, a random sample of $m$ predictors is chosen as split candidates from the full set of $p$ predictors. A fresh sample of $m$ predictors is taken at each split, and typically we choose $m ≈ \sqrt{p}$.

'Suppose that there is one very strong predictor in the data set, along with a num- ber of other moderately strong predictors. Then in the collection of bagged trees, most or all of the trees will use this strong predictor in the top split. Consequently, all of the bagged trees will look quite similar to each other. Hence the predictions from the bagged trees will be highly correlated. Un- fortunately, averaging many highly correlated quantities does not lead to as large of a reduction in variance as averaging many uncorrelated quan- tities.' - Random forests overcome this problem by forcing each split to consider only a subset of the predictors.

### Boosting
Idea: Trees are grown sequentially - each tree is grown using information from previously grown trees. 

---
**Algorithm: Boosting for Regression Trees**
1. Set $\hat{f(x)} = 0$ and $r_i = y_i$ for all $i$ in the training set.  

2. For $b = 1,2,\ldots,B$, repeat:  
   a. Fit a tree $\hat{f}^{\,b}$ with $d$ splits ($d+1$ terminal nodes) to the training data $(X, r)$.  
   b. Update $\hat{f}$ by adding in a shrunken version of the new tree:  
      $
      \hat{f}(x) \leftarrow \hat{f}(x) + \lambda \hat{f}^{\,b}(x)
      $  
   c. Update the residuals:  
      $
      r_i \leftarrow r_i - \lambda \hat{f}^{\,b}(x).
      $  

3. Output the boosted model:  
   $
   \hat{f}(x) = \sum_{b=1}^B \lambda \hat{f}^{\,b}(x)
   $
---

### Bayesian Additive Regression Trees
Idea: Each tree is constructed in a random manner as in bagging and random forests, and each tree tries to capture signal not yet accounted for by the current model, as in boosting.

---
**Algorithm:  Bayesian Additive Regression Trees (BART)**

1. Initialize  

$$
\hat f^{\,1}_1(x) = \hat f^{\,1}_2(x) = \cdots = \hat f^{\,1}_K(x) = \frac{1}{n} \sum_{i=1}^n y_i
$$

2. Compute the initial fit  

$$
\hat f^{\,1}(x) = \sum_{k=1}^K \hat f^{\,1}_k(x) = \frac{1}{n} \sum_{i=1}^n y_i
$$

3. For $b = 2, \ldots, B$:  

   (a) For $k = 1,2,\ldots,K$:  

   i. For $i = 1,\ldots,n$, compute the current partial residual  

   $$
   r_i = y_i - \sum_{k' < k} \hat f^{\,b}_{k'}(x_i) - \sum_{k' > k} \hat f^{\,b-1}_{k'}(x_i)
   $$  

   ii. Fit a new tree $\hat f^{\,b}_k(x)$ to $r_i$, by randomly perturbing the $k$-th tree from the previous iteration, $\hat f^{\,b-1}_k(x)$.  
   Perturbations that improve the fit are favored.  

   (b) Compute the updated ensemble  

   $$
   \hat f^{\,b}(x) = \sum_{k=1}^K \hat f^{\,b}_k(x)
   $$

4. After $L$ burn-in samples, compute the posterior mean  

$$
\hat f(x) = \frac{1}{B-L} \sum_{b=L+1}^B \hat f^{\,b}(x)
$$

---